In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

# Simple Linear Regression
A straight-line fit is a model of the form *y = ax + b* where *a* is commonly known as the slope, and *b* is the intercept.

In [ ]:
from scipy.stats import norm
norm.cdf(0.6772)

In [ ]:
rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
y = 2 * x - 5 + rng.randn(50)
plt.scatter(x, y);

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True) ## init
model.fit(x[:, np.newaxis], y) ## fit
xfit = np.linspace(0, 10, 1000)
yfit = model.predict(xfit[:, np.newaxis]) ## predict

plt.scatter(x, y)
plt.plot(xfit, yfit);

In [ ]:
model.coef_

In [ ]:
model.intercept_

In [ ]:
rng = np.random.RandomState(1)
X = 10 * rng.rand(100, 3)
y = 0.5 + np.dot(X, [1.5, -2., 1.])
model.fit(X, y)
print(model.intercept_)
print(model.coef_)

# Advertising Data
Explore

In [ ]:
import pandas as pd
adv = pd.read_csv('../_input/islr/Advertising.csv', usecols=(1,2,3,4))

In [ ]:
adv

We predict Sales by using the spending for TV, Radio and Newspaper.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2,2)

sns.regplot(x='TV',y='Sales', data=adv, ax=ax[0,0])
sns.regplot(x='Radio',y='Sales', data=adv, ax=ax[0,1])
sns.regplot(x='Newspaper',y='Sales', data=adv, ax=ax[1,0]);

We use sklearn to get the slope and intercept of the data between TV and Sales.

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np
lm = LinearRegression() # init
xfit = lm.fit(adv['TV'].values[:, np.newaxis], adv['Sales'].values) # fit

In [ ]:
xfit.coef_

In [ ]:
xfit.intercept_

Graph the line into the dataset to see how it fits.

In [ ]:
sns.regplot(x='TV', y='Sales', data=adv)
plt.plot([0, 250],[xfit.predict([[0]]), xfit.predict([[250]])]);

If X is at zero, then the value of y would below and this is the intercept.

In [ ]:
xfit.predict([[0]])

If X is 5 and 250, then we get the corresponding values for Y. Add a line between these points and you get the regression line.

But how are these coefficients being calculated?

In [ ]:
[xfit.predict([[5]]), xfit.predict([[250]])]

# Estimating the Coefficients
I want to calculate the intercept and slope manually so that I can understand the concept of it.

In [ ]:
x = adv['TV']
y = adv['Sales']

There are two components to the slope. First is the sum distance of each point of x away from its means multiplied by distance of y from its mean. The denominator is the squared distance of x from its mean. This formula calculates the slope where RSS is minimized as small as possible.

In [ ]:
## Breaking down the slope
distance_coefficient = sum((x - x.mean()) * (y - y.mean()))
spread_x = sum((x - x.mean())**2)
slope =  distance_coefficient / spread_x
slope

In [ ]:
# b0 is the intercept
intercept = y.mean() - slope * x.mean()
intercept

The intercept is the starting point of the regression line. Remember that the regression line is the average of x and y. We get it by subtracting the product of slope and the x.mean by y mean.

In [ ]:
yhat = intercept + slope * adv['TV']
yhat

Now that we know the intercept and slope, we can use the points of x to estimate the value of y by adding intercept and multiplying slope with all the points of x.

In [ ]:
sns.relplot(x=x, y=y, alpha=.3)
sns.lineplot(x=x, y=yhat, color='red');

We get the regression line - the line that has the minimum distance from all the points in our data.

# Assessing the Accuracy of the Coefficient Estimates

The regression line is based on the observed data. Which means that is only an estimate of truth based on what we currently know. There is a chance that the line would not exactly match the truth. So, instead of having a line estimate, we create a boundary around that line where we suspect that the true population line exists, which is called the Confidence Interval.

If we ran a 100 trials, 95 of those would be within the confidence interval. And if you average all of those 100 trials, the outcome would be very close to the truth.

In [ ]:
print(intercept, slope)

The confidence interval are just the standard deviations. A 95% confidence interval is around +- 1.98 standard deviations away from the mean. 
> The term SD is usually reserve for the population data. So instead, we use the term Standard Error or SE for the estimated data. But essentially, it is the same as the sd but for the estimated data.

In [ ]:
rss = sum((y - yhat)**2)
rss

To calculate of SE, we use the RSS or the sum of the squared distance between all the points away from yhat or the regression line that we made. It is essentially the variance of the estimated data. To get the SE, we square root the variance.

In [ ]:
n = len(adv)
rse = np.sqrt((rss/(n-2)))
rse

But because this is an estimate, we are making an assumption therefore we divide by n - 2 degrees of freedom. The result is the RSE or Residual Standard Error.

In [ ]:
yhat.name = 'target'
se_slope = np.sqrt(rse**2 / sum( (x - x.mean())**2 ) )
se_slope

In [ ]:
se_intercept = np.sqrt(
    rse**2 * ( 1/n + (x.mean()**2 / sum((x - x.mean())**2))  ))
se_intercept

In [ ]:
slope + se_slope * 2

In [ ]:
ci = 1.98
yhat_se_up = (
    (intercept + (ci*se_intercept)) + 
    (slope + (ci*se_slope)) *
    adv['TV'])
    
yhat_se_down = (
    (intercept -(ci*se_intercept)) +
    (slope - (ci*se_slope)) *
    adv['TV'])

In [ ]:
# sns.relplot(x=x, y=y, alpha=.3)
# sns.lineplot(x=x, y=yhat, color='red');
sns.regplot(x='TV', y='Sales', data=adv, n_boot=100)

sns.lineplot(x=x, y=yhat_se_up, color='green');
sns.lineplot(x=x, y=yhat_se_down, color='red');
sns.lineplot(x=x, y=yhat, color='orange');



## t-statistic
From the standard Error, we calculate the t-statistic which is the exact point from the standard deviation.

In [ ]:
(slope - 0) / se_slope

In [ ]:
(intercept - 0) / se_intercept

# Assessing the Accuracy of the Model
Quantify the extent to which the model fits the data.
- RSE 3.26
- R2 0.612
- F-statistic

In [ ]:
np.sqrt(sum((y - yhat)**2) / (n-2))

In [ ]:
tss = sum((y - y.mean())**2)
tss

TSS or Total Sum of Squares or Variance measures variability of Y before the regession is ran.

In [ ]:
print(sum((y - yhat)**2))
rss

In contrast, RSS measures the amount of variability that is left unexplained after performing the regression line.

In [ ]:
r2 = (tss - rss) / tss
r2

In [ ]:
sns.regplot(x='TV', y='Sales', data=adv, ci=False)
sns.regplot(x=np.zeros(n), y='Sales', data=adv, ci=False)

sns.lineplot(x=x, y=yhat, color='orange');
sns.lineplot(x=x, y=y.mean(), color='blue');

TSS - RSS measures the amount of variability in the response that is explained by performing the regression.

# Multiple Linear Regression

In [ ]:
target = adv['Sales']

In [ ]:
newspaper = adv['Newspaper']

In [ ]:
## Breaking down the slope
def findSlope(x, y):
    distance_coefficient = sum((x - x.mean()) * (y - y.mean()))
    spread_x = sum((x - x.mean())**2)
    slope =  distance_coefficient / spread_x
    return slope

In [ ]:
slope = findSlope(newspaper, target)
slope

In [ ]:
def findIntercept(x, y, slope):
    intercept = y.mean() - slope * x.mean()
    return intercept

In [ ]:
intercept = findIntercept(newspaper, target, slope)
intercept

In [ ]:
yhat = intercept + slope * newspaper

In [ ]:
sns.regplot(x='Newspaper', y='Sales', data=adv)
sns.lineplot(x=newspaper, y=yhat);

In [ ]:
rss = sum((target - yhat)**2)
rss

In [ ]:
rse = np.sqrt((rss / (n - 2)))
rse

In [ ]:
se_slope = np.sqrt(rse**2 / sum( (newspaper - newspaper.mean())**2 ) )
se_slope

In [ ]:
se_intercept = np.sqrt(
    rse**2 * ( 1/n + (newspaper.mean()**2 / sum((newspaper - newspaper.mean())**2))  ))
se_intercept

In [ ]:
t_stat_intercept = (intercept - 0) / se_intercept
t_stat_intercept

In [ ]:
t_stat_slope = (slope - 0) / se_slope
t_stat_slope

In [ ]:
from scipy.stats import norm
print(1-norm.cdf(t_stat_intercept), 1-norm.cdf(t_stat_slope))

## Now do the Automated way

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm = LinearRegression()
xfit = lm.fit(newspaper.values[:, np.newaxis], target.values)
print(xfit.coef_, xfit.intercept_)

For every 1000 usd we spend on Newspaper, we sell around 55 more units.

In [ ]:
lm = LinearRegression()
xfit = lm.fit(adv['Radio'].values[:, np.newaxis], target.values)
print(xfit.coef_, xfit.intercept_)

For every 1000 usd we spend on Radio, we sell around 202 more units.

In [ ]:
adv.corr()

In [ ]:
lm = LinearRegression()
xfit = lm.fit(adv[['TV', 'Radio', 'Newspaper']].values, target.values)
print(xfit.coef_, xfit.intercept_)

In [ ]:
yhat = (xfit.intercept_ +
        xfit.coef_[0] * adv['TV'] +
        xfit.coef_[1] * adv['Radio'] +
        xfit.coef_[2] * adv['Newspaper'] )

In [ ]:
rss = sum((target - yhat)**2)
rss

In [ ]:
tss = sum((target - target.mean())**2)
tss

In [ ]:
f_stat = ((tss - rss)/3) / (rss/(n-3-1))
f_stat

the large F-statistic suggests that at least one of the advertising media must be related to sales.

In [ ]:
r2 = (tss-rss)/tss
r2

In [ ]:
rse = np.sqrt((rss / (n - 3 -1)))
rse